In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
from pettingzoo.classic import connect_four_v3
import numpy as np
import matplotlib.pyplot as plt

import os
os.environ["SDL_VIDEODRIVER"] = "dummy"
from IPython.display import clear_output
import random

In [5]:
#Initialisation de l'environnement
env = connect_four_v3.env(render_mode="rgb_array")

env.reset()

# The main difference with the standard gym api is the way the environment is queried. The `step` method return `None`. To get the data on the environment, use the `last` method
state, reward, terminated, truncated, info = env.last()

# state is a dictionary with two keys: observation and action_mask
print(
    state["observation"].shape
)  # Observation is a numpy array with three coordinates, indicating the positions of the pieces of of player 0 and 1 on the the board
print(state["observation"][:, :, 0])  # Where the pieces of player 0 are
print(state["observation"][:, :, 1])  # Where the pieces of player 1 are

print(state["action_mask"])  # an array showing whether the actions are legal or nots


(6, 7, 2)
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
[1 1 1 1 1 1 1]


<h4>Brouillon<h4>

In [91]:
#env.reset()
env.step(3)

state, reward, terminated, truncated, info = env.last()

print(
    state["observation"].shape
) 
print(state["observation"][:, :, 0])  
print(state["observation"][:, :, 1])  

print(state["action_mask"])

(6, 7, 2)
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0]]
[1 1 1 1 1 1 1]


In [74]:
tab=state["observation"][:, :, 1]
tab

array([[0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0]], dtype=int8)

In [93]:
act=3
for i in range(height):
    if state["observation"][:, act, 1][i] != 0:
        print(i)
        break

3


In [30]:
state, reward, terminated, truncated, info = env.last()
reward

0

<h3>Local view of the game (the last move)<h3>

Première approche (naive) : Q-Learning avec matrice d'état = Toutes les positions de la grille (**état d'un agent= coordonnées du dernier coût joué par ce même agent**)

In [18]:
state["observation"][4, 2, 1]

1

In [6]:
#Dimensions du tableau
height=6
width=7

#Fonction qui initialise la matrice Q
def initialize_Q(): #Initialise la matrice Q (que des zéros)
    return np.zeros((height, width, width)) 

In [16]:
class Player:
    def __init__(self, _id, Q, rng=None): 
        self.name = 'Player '+str(_id)
        self.Q=initialize_Q()
        self.st=None #Etat
        self._id=_id #Identifiant
        
    def get_action(self, epsilon=None):
        if (random.uniform(0,1)<epsilon) or (self.st is None): #Exploration ou premier coupde la partie
            action=random.randint(0,6)  
        else:
            (x,y)=self.st
            Q=self.Q
            action=np.argmax(Q[x,y]) #On choisit la meilleure action parmi celles qu'il peut jouer (meilleure dans le sens maximise Q)
            return action
    def highest_not_null(self, action, obs): #Fonction auxiliaire à la fonction pour déterminer l'état
        _id=self._id
        for i in range(height):
            if obs["observation"][:, act, _id][i] != 0: 
                return i

    def update_state(self,action, obs): #Fonction qui update l'état du joueur (trouve en fait le dernier coup joué)
        y=action
        x=self.highest_not_null(action, obs)
        self.st=(x,y)
    def update_Q(self, last_state, action): #fonction qui update Q selon 
        next_action=self.get_action(epsilon=0.0)
        (a,b)=self.st
        (x,y)=last_state
        Q=self.Q
        self.Q[x,y][action]=Q[x,y][action] + alpha*(r + gamma*Q[a,b][next_action]-Q[x,y][action])
        

In [8]:
arr=np.array([[[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 7., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]]])

In [9]:
arr[2,4]

array([0., 0., 0., 0., 0., 7., 0.])

In [18]:
def play_game(env, agent0, agent1, alpha, gamma ,r ,e, display=False): #e epsilon coeff d'exploration
    done = False
    env.reset()
    obs, _, _, _, _ = env.last()
    while not done:
        for i, agent in enumerate([agent0, agent1]):
            last_state=agent.st
            action = agent.get_action(obs, epsilon=e) #Epsilon = taux d'exploration
            env.step(action) #Préciser à l'environnement pettingZoo que l'action prise
            obs, reward, terminated, _, _ = env.last()
            agent.update_state(action, obs) #Mise à jour de l'état de l'agent
            if display:
                clear_output(wait=True)
                plt.imshow(env.render())
                plt.show()
            done = terminated #Déroulement fini
            #update Q
            agent.update_Q(last_state, action)
            
            if np.sum(obs["action_mask"]) == 0:
                if display: 
                    print('Draw')
                return 0.5
            if done:
                if display:
                    print(f"Player {i}: {agent.name} won")
                    print(obs['observation'][:, :, 0]- obs['observation'][:, :, 1])
                return i